In [1]:
!pip install gensim

You should consider upgrading via the '/Users/max/.pyenv/versions/3.7.11/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
!pip install nltk

You should consider upgrading via the '/Users/max/.pyenv/versions/3.7.11/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
!pip install texthero

You should consider upgrading via the '/Users/max/.pyenv/versions/3.7.11/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
!pip install sentence-transformers

You should consider upgrading via the '/Users/max/.pyenv/versions/3.7.11/bin/python3.7 -m pip install --upgrade pip' command.


In [5]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import tensorflow_hub as hub
from nltk.tokenize import word_tokenize
from scipy import spatial
import nltk
import texthero as hero
from sentence_transformers import SentenceTransformer
nltk.download('punkt')

/Users/max/.pyenv/versions/3.7.11/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
[nltk_data] Downloading package punkt to /Users/max/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

here we generate the dataframe, which keyword named as title and GT recommendation named as content.

We are using here just the NYT articles, as buy using articles from just one publisher we hope to get a better more consistant matching and less confusion, which is important, when building the pipeline

In [6]:
df = pd.read_csv('../comparison_dataset/articles1.csv')
df = df[df['publication'] == 'New York Times']
df = df.drop(columns = ['id', 'publication', 'author', 'date', 'year', 'month', 'url', 'Unnamed: 0'])
df

,title,content
0,House Republicans Fret About Winning Their Hea...,WASHINGTON — Congressional Republicans have...
1,Rift Between Officers and Residents as Killing...,"After the bullet shells get counted, the blood..."
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...","When Walt Disney’s “Bambi” opened in 1942, cri..."
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...","Death may be the great equalizer, but it isn’t..."
4,Kim Jong-un Says North Korea Is Preparing to T...,"SEOUL, South Korea — North Korea’s leader, ..."
...,...,...
7798,U.N. Relief Official Calls Crisis in Aleppo th...,The top aid official at the United Nations gav...
7799,Federal Judge Curbs Enforcement of North Carol...,A federal judge on Friday curbed the enforceme...
7800,Mexicans Accuse President of ‘Historic Error’ ...,MEXICO CITY — If President Enrique Peña Nie...
7801,"U.S. Presses for Truce in Syria, With Its Larg...","HANGZHOU, China — The image of a Syrian ..."


now we try to find embedding methods, which in an ideal matching, or we could also just try to minimize distance between the embedded title and content

In [7]:
df['content'] = hero.clean(df['content'])

In [8]:
def embedding_by_doc_2_vec(df):
  data = df['content']
  tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
  # df['tagged_data'] = tagged_data
  max_epochs = 5
  vec_size = 20
  alpha = 0.025

  model = Doc2Vec(#size = vec_size,
    vector_size=vec_size,
                  alpha=alpha, 
                  min_alpha=0.00025,
                  min_count=1,
                  dm =1)
    
  model.build_vocab(tagged_data)

  for epoch in range(max_epochs):
      print('iteration {0}'.format(epoch))
      model.train(tagged_data,
                  total_examples=model.corpus_count,
                  #epochs = model.iter
                  epochs=model.epochs)
      # decrease the learning rate
      model.alpha -= 0.0002
      # fix the learning rate, no decay
      model.min_alpha = model.alpha
  print('here')
  print(model.docvecs['1'])
  return model

In [9]:
model_d2v = embedding_by_doc_2_vec(df)

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4


KeyboardInterrupt: 

In [ ]:
#model_d2v.save("doc2vec_clean.model")
#model_d2v = Doc2Vec.load_doc("doc2vec_clean.model")

In [ ]:
def embedd_d2v(model, doc):
    return model.infer_vector(doc.split())

df['embed_title_d2v'] = df['title'].apply(lambda x: embedd_d2v(model_d2v, x))
df['embed_content_d2v'] = df['content'].apply(lambda x: embedd_d2v(model_d2v, x))
df

,title,content,embed_title_d2v,embed_content_d2v
0,House Republicans Fret About Winning Their Hea...,washington -- congressional republicans new fe...,"[0.06387405, 0.0027298941, -0.07726031, 0.0393...","[-0.1479157, 2.0191174, 1.9422089, -1.2744783,..."
1,Rift Between Officers and Residents as Killing...,bullet shells get counted blood dries votive c...,"[0.05206804, 0.035272043, -0.025424976, -0.022...","[8.846913, -1.4289446, 3.6064503, -2.3176916, ..."
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...","walt disney' ""bambi"" opened critics praised sp...","[0.064312935, 0.009777807, -0.016095767, -0.01...","[1.1507585, -0.75324535, -3.7713225, -3.423285..."
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",death may great equalizer ' necessarily evenha...,"[0.06839463, 0.01517621, -0.040933907, -0.0239...","[1.8920553, -0.74019766, -2.7524686, 1.0556654..."
4,Kim Jong-un Says North Korea Is Preparing to T...,seoul south korea -- north korea' leader kim s...,"[0.12724127, -0.01452525, -0.05128109, -0.0053...","[-0.9053595, 1.3270155, -0.028560245, 1.839707..."
...,...,...,...,...
7798,U.N. Relief Official Calls Crisis in Aleppo th...,top aid official united nations gave gloomy as...,"[0.12453497, -0.04603657, -0.029953754, 0.0265...","[-0.08758663, 0.97439015, 0.8159305, 0.1262278..."
7799,Federal Judge Curbs Enforcement of North Carol...,federal judge friday curbed enforcement north ...,"[0.066655576, -0.0291929, -0.0071223252, -0.01...","[2.723682, 1.3773351, -0.05944748, 0.9979264, ..."
7800,Mexicans Accuse President of ‘Historic Error’ ...,mexico city -- president enrique pena nieto in...,"[0.118740484, -0.028534405, -0.044770654, -0.0...","[-0.70808154, 2.3987489, -2.6012354, 1.5189507..."
7801,"U.S. Presses for Truce in Syria, With Its Larg...",hangzhou china -- image syrian boy dazed blood...,"[0.07130332, -0.02886117, -0.044939496, -0.033...","[-3.2483962, 1.4409665, 1.1384422, 1.4458574, ..."


In [15]:
def cosine_similarity(df):
    return 1 - spatial.distance.cosine(df[0], df[1])

In [ ]:
df['d2v_similarity'] = df[['embed_title_d2v', 'embed_content_d2v']].apply(cosine_similarity, axis = 1)
df['d2v_similarity'].mean()

0.39202946531022015

In [20]:
embed_USE = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def embed_with_USE(embedder , x):
    return tf.reduce_mean(embedder(x.split('.')),0)

df['embed_title_USE'] = df['title'].apply(lambda x: embed_with_USE(embed_USE, x))
df['embed_content_USE'] = df['content'].apply(lambda x: embed_with_USE(embed_USE, x))
df['USE_similarity'] = df[['embed_title_USE', 'embed_content_USE']].apply(cosine_similarity, axis = 1)
df['USE_similarity'].mean()


NotImplementedError: Cannot convert a symbolic Tensor (Mean:0) to a numpy array.

In [ ]:
model_BERT = SentenceTransformer(r"sentence-transformers/paraphrase-MiniLM-L6-v2")

def embed_with_USE(model , x):
    return model.encode(x)

df['embed_title_BERT'] = df['title'].apply(lambda x: embed_with_USE(model_BERT, x))
df['embed_content_BERT'] = df['content'].apply(lambda x: embed_with_USE(model_BERT, x))
df['BERT_similarity'] = df[['embed_title_BERT', 'embed_content_BERT']].apply(cosine_similarity, axis = 1)
df['BERT_similarity'].mean()

0.5456832346447217

In [10]:
!pip uninstall tensorflow -y
!pip uninstall tensorflow-cloud -y
!pip install -U tensorflow==1.15
!pip uninstall pytorch-lightning -y
!pip uninstall tensorflow-probability -y

Found existing installation: tensorflow 1.15.0
Uninstalling tensorflow-1.15.0:
  Successfully uninstalled tensorflow-1.15.0
  Using cached tensorflow-1.15.0-cp37-cp37m-macosx_10_11_x86_64.whl (124.0 MB)
You should consider upgrading via the '/Users/max/.pyenv/versions/3.7.11/bin/python3.7 -m pip install --upgrade pip' command.


In [11]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.load("https://tfhub.dev/google/elmo/2")

def create_elmo_embeddings(data):
    return_ls = []
    for dat in data.tolist():
        embed=elmo.signatures['default']( tf.convert_to_tensor([dat]))["elmo"]
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            sess.run(tf.tables_initializer())
            out_x=sess.run(embed)
            #out_y=ses.run(tf.reduce_mean(embed,1))
        return_ls.append(out_x)
    return return_ls

temp1 = create_elmo_embeddings(df['content'][:5])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2022-04-16 19:42:03.989966: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-04-16 19:42:04.041330: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fde2d3f2150 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-04-16 19:42:04.041346: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version



In [18]:
import tensorflow as tf
import tensorflow_hub as tf_hub
elmo = tf_hub.Module("https://tfhub.dev/google/elmo/2")
embeddings = elmo(
    ["the cat is on the mat", "dogs are in the fog"],
    signature="default",
    as_dict=True)["elmo"]
embeddings

ValueError: tf.enable_eager_execution must be called at program startup.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

max_feat = 10000

tfidf = TfidfVectorizer(min_df=5, max_features=max_feat, ngram_range=(1,3),
                        strip_accents='unicode',
                        lowercase =True, analyzer='word', token_pattern=r'\w+',
                        use_idf=True, smooth_idf=True, sublinear_tf=True, 
                        stop_words = 'english')
svdT = TruncatedSVD(n_components=390)
tfidf.fit(df['content'])
df['embed_content_tfidf'] = svdT.fit_transform(tfidf.transform(df['content'])).tolist()
df['embed_title_tfidf'] = svdT.fit_transform(tfidf.transform(df['title'])).tolist()
df['tfidf_similarity'] = df[['embed_title_tfidf', 'embed_content_tfidf']].apply(cosine_similarity, axis = 1)
df['tfidf_similarity'].mean()


NameError: name 'cosine_similarity' is not defined

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

import os
root_folder='.'
data_folder_name='data'

DATA_PATH = os.path.abspath(os.path.join(root_folder, data_folder_name))

glove_filename='glove.6B.100d.txt'
#glove_path = os.path.abspath(os.path.join(DATA_PATH, glove_filename))
#glove_input_file = glove_filename
#word2vec_output_file = glove_filename+'.word2vec'
#glove2word2vec(glove_path, word2vec_output_file)

# load the Stanford GloVe model
word2vec_output_file = glove_filename+'.word2vec'
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [ ]:
import numpy as np
def embed_glove(s):
    return_ls = []
    for sentence in s:
        array = []
        for word in sentence.split():
            try:
                array.append(model.get_vector(word))
            except:
                pass
        array = np.array(array)
        #print(array.shape)
        return_ls.append(array.mean(axis = 0))

    return return_ls
#Show a word embedding
df['embed_content_glove'] = embed_glove(df['content'])
df['embed_title_glove'] = embed_glove(df['title'])
df['glove_similarity'] = df[['embed_title_glove', 'embed_content_glove']].apply(cosine_similarity, axis = 1)
df['glove_similarity'].mean()

/Users/max/.pyenv/versions/3.7.11/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: Mean of empty slice.
  del sys.path[0]


0.8266875235549795

In [14]:
from gensim.models import FastText

sentences = df['content']
sentences = [s.split() for s in sentences]
model = FastText(sentences)

KeyboardInterrupt: 

In [ ]:
def embed_fasttext(s):
    return_ls = []
    for sentence in s:
        array = []
        for word in sentence.split():
            try:
                array.append(model.wv[word])
            except:
                pass
        array = np.array(array)
        #print(array.shape)
        return_ls.append(array.mean(axis = 0))

    return return_ls

df['embed_content_fasttext'] = embed_fasttext(df['content'])
df['embed_title_fasttext'] = embed_fasttext(df['title'])
df['fasttext_similarity'] = df[['embed_title_fasttext', 'embed_content_fasttext']].apply(cosine_similarity, axis = 1)
df['fasttext_similarity'].mean()